In [ ]:
#!pip install newsapi-python

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_ollama import ChatOllama
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import get_buffer_string, AIMessage, HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from typing import Dict
import networkx as nx  # For graph memory (pip install networkx if needed)

llm = ChatOllama(model="mistral", temperature=0.5)
store: Dict[str, BaseChatMessageHistory] = {}
config = {"configurable": {"session_id": "rahul_trip"}}

prompt = ChatPromptTemplate.from_template(
    "Travel bot: Use history.\n{history}\nHuman: {input}\nBot:"
)
base_chain = prompt | llm

# 🎒 Stateful Chats: Introducing Memory in LangChain

Imagine building a travel bot for Rahul: "Plan Mumbai trip." → "Budget?" → "Beaches?" Without memory, Mistral forgets—each query starts fresh, like amnesia. *Enter memory*: It persists history across turns, injecting context into prompts for natural flow. In LangChain 0.4+, no deprecated hacks—just `RunnableWithMessageHistory` wrapping your LCEL chain (prompt | LLM). 

Why care? 70% of AI apps are conversational—memory cuts hallucinations, boosts engagement. We'll demo 4 types in our "Rahul's Trip" scenario: From raw buffers (exact but bulky) to graphs (smart relations). Each plugs in: Define `get_history(session_id)` → Wrap chain → Invoke with `config={"session_id": "rahul_trip"}`.

Key Insight: Memory's a *store* (dict of histories) + *injector* (`{history}` in prompt). Start simple—buffer for basics—then layer for realism. 

*Pro Tip*: Sessions scope via ID (user/email). Prod? Swap in-memory for Redis/Postgres.

def get_buffer_history(session_id: str):: Factory function—what? Returns a history object per session (e.g., "rahul_trip"). 
Why? Scopes conversations (multi-user safe).

if session_id not in store: store[session_id] = InMemoryChatMessageHistory(): 

Checks dict store (global from setup). If missing, creates new InMemoryChatMessageHistory—what? A list-like object for messages. Why import it? From langchain_core.chat_history—core primitive for message persistence (HumanMessage/AIMessage classes).

return store[session_id]: Hands back the mutable history—wrapper modifies it.

buffer_chain = RunnableWithMessageHistory(...): Wraps base_chain (your prompt | LLM). What? Creates a new runnable that adds history magic on invoke. 

Why import RunnableWithMessageHistory? From langchain_core.runnables—LCEL's way to make chains stateful without boilerplate.

base_chain: Your core pipe (prompt | llm)—why? Memory layers on top, not replaces.
get_buffer_history: Callback to fetch history—why? Enables custom stores (e.g., DB later).
input_messages_key="input": Maps user input to HumanMessage.
history_messages_key="history": Maps loaded history to prompt var.

queries = [...]: List of turns—what? Simulates multi-turn chat. Why? Shows buildup.

for q in queries: resp = buffer_chain.invoke({"input": q}, config=config): Invokes wrapped chain—what? Loads history, runs base_chain, appends new msgs. config: Dict with session_id—why? Scopes to "rahul_trip".

print(f"Bot: {resp.content[:100]}..."): Truncates response—why? Clean output.
print(f"History len: {len(store['rahul_trip'].messages)}"): Inspects store—what? Proves persistence (e.g., 6 msgs: 3 human + 3 AI).

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory

def get_buffer_history(session_id: str):
    if session_id not in store: store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

buffer_chain = RunnableWithMessageHistory(base_chain, get_buffer_history, 
                                          input_messages_key="input", 
                                          history_messages_key="history")

# Demo: 3 turns
queries = ["Plan Mumbai trip.", "Budget: 50k INR.", "Include beaches?"]
for q in queries:
    resp = buffer_chain.invoke({"input": q}, config=config)
    print(f"Bot: {resp.content[:100]}...")  # Truncated
print(f"History len: {len(store['rahul_trip'].messages)}")

Bot: 1. Research popular tourist attractions in Mumbai such as the Gateway of India, Chhatrapati Shivaji ...
Bot: 1. Research budget-friendly accommodations in the Colaba or Marina Drive area that fit within your b...
Bot: 1. Research popular beaches in Mumbai such as Juhu Beach and Chowpatty Beach that can be added to yo...
History len: 6


In [12]:
store['rahul_trip'].messages

[HumanMessage(content='Plan Mumbai trip.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='1. Research popular tourist attractions in Mumbai such as the Gateway of India, Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (Prince of Wales Museum), Marine Drive, and Elephanta Caves.\n2. Check for flights to Chhatrapati Shivaji International Airport in Mumbai from your departure city.\n3. Book accommodations in a central location such as Colaba or Marina Drive area.\n4. Plan the itinerary for each day of your trip, ensuring that you have enough time to visit all the attractions and also account for travel time between locations.\n5. Research local transportation options such as taxis, auto-rickshaws, and the Mumbai Metro.\n6. Familiarize yourself with local customs and etiquette, as well as safety tips specific to Mumbai.\n7. Plan meals at popular restaurants in Mumbai for a taste of local cuisine.\n8. Check the weather forecast for your travel dates and pack accordingly.\n9

class SummaryHistory(BaseChatMessageHistory):: Extends base—what? Inherit load/add methods. Why import BaseChatMessageHistory? From langchain_core.chat_history—abstract for custom (e.g., file/DB).
__init__: Sets session, llm (for summarizing), empty lists—why? State per instance.

add_messages(self, messages):: Override—what? Appends raw, triggers summary every 3. Why? Wrapper calls this post-response.

self.full_history.extend(messages): Keeps all—why? Fallback if needed.
if len(...) % 3 == 0: Conditional—why? Batch for efficiency (tune to %5).
sum_prompt = ... | self.llm | (lambda x: x.content): LCEL sub-chain—what? Template to summarize, pipe LLM, extract content. Why? Reusable.
self.summary = sum_prompt.invoke({"chat": get_buffer_string(self.full_history)}): Invokes—what? Formats history string (from langchain_core.messages—why import? Utils like get_buffer_string). Updates summary.

messages(self):: Override—what? Returns recent 2 + summary as AIMessage. Why? Wrapper formats this into {history}—keeps prompt short.

The rest (get_summary_history, summary_chain, invoke) mirrors buffer—why? Reusability; just swap factory.

In [ ]:
class SummaryHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, llm):
        self.session_id = session_id; self.llm = llm; self.full_history = []; self.summary = ""

    def add_messages(self, messages):
        self.full_history.extend(messages)
        if len(self.full_history) % 3 == 0:  # Summarize every 3
            sum_prompt = ChatPromptTemplate.from_template("Summarize trip plan: {chat}") | self.llm | (lambda x: x.content)
            self.summary = sum_prompt.invoke({"chat": get_buffer_string(self.full_history)})

    def messages(self):
        return self.full_history[-2:] + [AIMessage(content=f"Summary: {self.summary}")]

def get_summary_history(session_id: str):
    if session_id not in store: store[session_id] = SummaryHistory(session_id, llm)
    return store[session_id]

summary_chain = RunnableWithMessageHistory(base_chain, get_summary_history, input_messages_key="input", history_messages_key="history")

# Demo
resp = summary_chain.invoke({"input": "Add flight to Delhi."}, config=config)
print("Bot (with summary):", resp.content)

__init__: Adds self.entities = {}—what? Dict for extracted facts (e.g., {"pref": "AC"}). Why? Structured recall.

add_messages: Extracts per batch—what? Sub-chain parses to JSON. Why JsonOutputParser (from langchain_core.output_parsers)? Auto-structures LLM output as dict.
self.entities.update(new_ents): Merges—why? Accumulates (overwrite duplicates).

In [ ]:
class EntityHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, llm):
        self.session_id = session_id; self.llm = llm; self.full_history = []; self.entities = {}

    def add_messages(self, messages):
        self.full_history.extend(messages)
        ent_prompt = ChatPromptTemplate.from_template('Extract JSON: {"pref": "desc"} from: {chat}') | self.llm | JsonOutputParser()
        new_ents = ent_prompt.invoke({"chat": get_buffer_string(messages)})
        self.entities.update(new_ents)

    def messages(self):
        ent_str = "\n".join(f"{k}: {v}" for k,v in self.entities.items())
        return self.full_history[-2:] + [AIMessage(content=f"Entities: {ent_str}")]

def get_entity_history(session_id: str):
    if session_id not in store: store[session_id] = EntityHistory(session_id, llm)
    return store[session_id]

entity_chain = RunnableWithMessageHistory(base_chain, get_entity_history, input_messages_key="input", history_messages_key="history")

# Demo
resp = entity_chain.invoke({"input": "Rahul prefers AC hotels, hates delays."}, config=config)
print("Bot:", resp.content)
print("Entities:", store['rahul_trip'].entities)  # {'pref': 'AC hotels, no delays'}

In [7]:
from typing import Dict
from langchain_core.chat_history import BaseChatMessageHistory

# Re-define store (from shared setup)
store: Dict[str, BaseChatMessageHistory] = {}
print("✅ Store re-defined!")

✅ Store re-defined!


In [ ]:
store.clear()  # Resets store—fresh for graph

Entity is flat—graph adds edges (e.g., Rahul → Mumbai:rainy). Why? Multi-hop (e.g., "Rainy prefs? Traverse").

add_messages: Scans for keywords, adds edges—what? Mock extraction (real: LLM triples). Why? Builds web incrementally.

messages: Stringifies edges—what? Prompt sees relations. Why? LLM reasons over "Rahul --trip_to--> Mumbai (rainy)".

In [8]:
class GraphHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str):
        self.session_id = session_id; self.full_history = []; self.graph = nx.DiGraph()  # Simple graph

    def add_messages(self, messages):
        self.full_history.extend(messages)
        # Mock extraction: Add nodes/edges (use LLM for real)
        for msg in messages:
            if "Mumbai" in msg.content: self.graph.add_edge("Rahul", "Mumbai", rel="trip_to", attr="rainy")
            if "indoor" in msg.content: self.graph.add_edge("Mumbai", "sites", rel="prefers", attr="museums")

    def messages(self):
        graph_str = "\n".join(f"{u} --{d['rel']}--> {v} ({d['attr']})" for u,v,d in self.graph.edges(data=True))
        return self.full_history[-1:] + [AIMessage(content=f"Graph: {graph_str}")]

def get_graph_history(session_id: str):
    if session_id not in store: store[session_id] = GraphHistory(session_id)
    return store[session_id]

graph_chain = RunnableWithMessageHistory(base_chain, get_graph_history, input_messages_key="input", history_messages_key="history")

# Demo
resp = graph_chain.invoke({"input": "Suggest indoor sites for rainy Mumbai."}, config=config)
print("Bot (graph-aware):", resp.content)
print("Graph edges:", list(store['rahul_trip'].graph.edges(data=True)))

NameError: name 'RunnableWithMessageHistory' is not defined

# 🔧 Empower Your Bot: Tools for Real-World Actions

Memory recalls *past*—tools fetch *now*. Picture Rahul's bot: "Mumbai news?" Mistral can't Google—hallucinates "Sunny skies!" Tools fix that: Bind functions (e.g., API calls) so LLM *decides* when to use them ("Reason: Need live data → Call get_travel_news"). Output? Structured `tool_calls` JSON—parse, execute, feed back.

In 0.4+, it's LCEL-pure: `@tool def func(args):` → `llm.bind_tools([func])` → Chain invoke. Why realistic? No mocks—connect NewsAPI (free headlines on "Mumbai travel"). Handles errors, parses JSON—feels prod-ready.

Demo Flow: Query → LLM suggests call → We run tool → Mock final response. Tease: This + memory = "Rahul, recall your prefs + breaking flood news = Reroute to Delhi?"

Big Picture: Tools = LLM's "hands." Start with one (news)—add weather/flights for multi-tool power. Ethical note: APIs cost/query limits—teach retries. Run the cell: Watch live headlines pop (Nov 11, 2025—monsoon vibes?). What's a tool you'd build?

*Pro Tip*: For loops (full resolution), use agents later. Here: Simple invoke for intro.

In [4]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage, AIMessage
from newsapi import NewsApiClient  # From pip above
import os

NEWS_API_KEY = "your_newsapi_key"  # Free signup
newsapi = NewsApiClient(api_key="0858b0cfd7614896a0b653227dad1792")

@tool
def get_travel_news(city: str, category: str = "general") -> str:
    """Fetch latest news for travel in a city (e.g., alerts, events)."""
    try:
        articles = newsapi.get_everything(q=f"{city} travel", language='en', sort_by='relevancy', page_size=3)
        if articles['articles']:
            news = "\n".join([f"- {a['title']}: {a['description'][:100]}..." for a in articles['articles']])
            return f"Latest {city} travel news: {news}"
        return f"No recent {city} travel news."
    except Exception as e:
        return f"News fetch error: {str(e)}"

# Bind to LLM + simple chain (no full loop—invoke direct)
llm_with_news = llm.bind_tools([get_travel_news])
tool_chain = ChatPromptTemplate.from_template("Travel bot with news: {input}") | llm_with_news

# Demo: LLM calls tool
resp = tool_chain.invoke({"input": "Rahul's Mumbai trip—any news alerts?"})
print("Raw resp:", resp.content)  # May include tool call

# Quick execute (for demo—parse & run)
if resp.tool_calls:
    tool_res = get_travel_news.invoke(resp.tool_calls[0]['args'])
    print("Tool result:", tool_res)
    # Feed back: final_resp = llm.invoke(f"News: {tool_res}\nRespond: {resp.content}")
    print("Final (mock): Based on news, check for floods in Mumbai.")

Raw resp: 
Tool result: Latest Mumbai travel news: - Why the Indian passport is falling in global ranking: Indians can travel to more visa-free destinations than a decade ago, but India's passport ranking ha...
- I moved back to my home city after a year abroad. Exploring my hometown like a tourist made the transition easier.: Moving home after living abroad was hard, so I embraced a new attitude and explored my hometown like...
- 70-km tunnel linking Coastal Road, BKC bullet train station, Mumbai Airport: MMRDA begins work on DPR for project to reduce congestion: The initiative aligns with MMRDA's broader plans for underground corridors to improve Mumbai's traff...
Final (mock): Based on news, check for floods in Mumbai.


# 📚 RAG Mastery: From Hallucinations to Grounded Answers (4-Hour Deep Dive)

Welcome to RAG—Retrieval-Augmented Generation—the secret sauce turning Mistral from "chatty guesser" to "doc-savvy expert." In our Rahul's Mumbai bot: No RAG? "Beaches are free!" With? Pulls real PDF facts: "Juhu Beach, 2k entry, AC lounges."

**The Flow**: Docs → Split chunks → Embed (vectors) → Store (FAISS) → Query embed → Retrieve top-k → Prompt stuff → Generate. We'll use local FAISS (no keys)—swap to Chroma later.

*Pro Tip*: Embeddings = "Math fingerprints" of text (cosine sim finds matches). Let's embed Mumbai magic!

In [ ]:
#!pip install langchain-text-splitters

In [ ]:
pwd

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Define embeddings (local, offline model—downloads ~80MB first time)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("✅ Embeddings loaded!")

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Load & chunk
loader = TextLoader("mumbai_guide.txt")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = splitter.split_documents(docs)

# Embed & retrieve
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LCEL chain
prompt = ChatPromptTemplate.from_template("Answer from context: {context}\nQuestion: {question}")
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Demo
print(chain.invoke("Budget AC beaches in Mumbai?"))
# Prints retrieved chunks + grounded answer

 In Mumbai, the budget AC beach option mentioned is Juhu Beach. The cost for an AC beach lounge with drinks there is 2000 INR for shaded cabanas. Another budget beach option, though not specifically mentioned as having AC facilities, is Versova Beach. However, it's important to note that the information provided doesn't explicitly state that these beaches have air-conditioned facilities on the premises.


In [ ]:
#!pip install langchain-experimental

In [18]:
# Advanced chunking: Semantic (try langchain_experimental for better)
from langchain_experimental.text_splitter import SemanticChunker  # pip install langchain-experimental

semantic_splitter = SemanticChunker(embeddings)  # Breaks on meaning shifts
splits_semantic = semantic_splitter.split_documents(docs)
vectorstore_sem = FAISS.from_documents(splits_semantic, embeddings)
retriever_sem = vectorstore_sem.as_retriever(k=3)

chain_sem = (
    {"context": retriever_sem, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Semantic chunks:", chain_sem.invoke("Rainy day spots?"))
# Better: Full sentences, not mid-para cuts

Semantic chunks:  In Mumbai, the rainy day indoor spots are the Prince of Wales Museum (500 INR entry) and Chhatrapati Shivaji Maharaj Vastu Sangrahalaya. These places offer AC halls with Mughal artifacts and ancient sculptures, as well as world-class exhibits respectively. Another option is visiting the Gateway of India, where you can find an iconic arch (free exterior) and take a boat to Elephanta Caves (200 INR ferry, cave tickets 40 INR). Keep in mind that during the rainy season (June-Sep), it's essential to pack umbrellas.


In [ ]:
#pip install langchain-community

In [1]:
from langchain_community.retrievers import MultiQueryRetriever  # Community path—key!

ImportError: cannot import name 'MultiQueryRetriever' from 'langchain_community.retrievers' (e:\LTI\ollama-env\Lib\site-packages\langchain_community\retrievers\__init__.py)

In [19]:
from langchain.retrievers.multi_query import MultiQueryRetriever

mq_retriever = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm)
chain_mq = (
    {"context": mq_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain_mq.invoke("Cheap lounges?"))  # Rewrites to "affordable beach spots OR budget AC..."

ModuleNotFoundError: No module named 'langchain.retrievers'

In [ ]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

compressor = LLMChainExtractor.from_llm(llm)
comp_retriever = ContextualCompressionRetriever(base_retriever=retriever, base_compressor=compressor)

chain_comp = (
    {"context": comp_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain_comp.invoke("Beaches—ignore history."))  # Prunes irrelevant chunks

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splits)
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, bm25_retriever], weights=[0.7, 0.3])

chain_hybrid = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain_hybrid.invoke("Juhu Beach exact details?"))  # Vectors for sim + BM25 for keywords

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from datasets import Dataset  # pip install datasets

# Sample eval dataset (mock 3 queries/answers)
data = {
    "question": ["Budget beaches?"],
    "answer": [chain.invoke("Budget beaches?")],
    "contexts": [[doc.page_content for doc in retriever.invoke("Budget beaches?")]],
    "ground_truth": ["Juhu: 2k, AC lounges"]  # Your "gold" answers
}
ds = Dataset.from_dict(data)

scores = evaluate(ds, metrics=[faithfulness, answer_relevancy])
print("Faithfulness:", scores['faithfulness'])  # >0.8 = No hallucinations

In [ ]:
# From memory: Assume entity_history.entities = {'budget': '50k'}
def filtered_retrieve(query, entities):
    # Mock filter: Add to query
    filtered_q = f"{query} under {entities.get('budget', 'any')}"
    return retriever.invoke(filtered_q)

# Use in chain
def rag_with_entities(input_q, config):
    ents = store[config["configurable"]["session_id"]].entities
    filtered = filtered_retrieve(input_q, ents)
    return prompt.format(context="\n".join([c.page_content for c in filtered]), question=input_q) | llm

print(rag_with_entities("AC beaches?", config))  # Filters by budget entity

In [2]:
from langchain.retrievers.multi_query import MultiQueryRetriever  # Core langchain path

ModuleNotFoundError: No module named 'langchain.retrievers'

In [3]:
from langchain.retrievers.multi_query import MultiQueryRetriever
print("Success:", MultiQueryRetriever)

ModuleNotFoundError: No module named 'langchain.retrievers'